## Transferring images from the Postgres DB into the Digital Ocean Space

Here's what we need to do to transfer over the raw image data so it's no longer in the database.

1. Determine that the size of the DO Space we've chosen will be big enough to last a long time. What is the current size of the images in the DB? How fast do we expect the space usage to grow, given reasonable assumptions about how many keywords we're using?
2. Get a list of all images we're going to query for
3. Update the API so that saving all new images will save to the DO Space rather than the database
4. Get the image binary data of an image, save it with the new API image save endpoint, then check that any query that includes that image metadata links to the new DO Space image URL
5. run for all images

### 1. size of Space

I'll check back on this after transfering over all the images from the DB.

### 2. list of all images to query

I have node running the server.js file in the api folder on my test server, which has the same database as prod.

In [1]:
# No, I don't want to implement log levels
VERBOSE_MODE=False
SHORT=False
RANGE=(None,None)
# RANGE=(0,100)
# RANGE=(100,None)
num_worker_threads = 50

In [2]:
import boto3
import io
import json
import os
import requests
from PIL import Image
import random
import imagehash
import queue
import threading
import time
import imghdr
import os
import ipyplot
from collections import defaultdict

# base = 'http://159.89.80.47'
base = 'http://api.firewallcafe.com'
folder = 'images'

try:
    os.mkdir(folder)
except FileExistsError:
    pass

In [3]:
# pages = {}
# for i in range(5000):
#     img_page = requests.get(base + f'/images?page={i}')
#     pages[i] = img_page.json()
#     if len(pages[i]) == 0:
#         break
#     if i%100 == 0:
#         print(i, pages[i][0]['image_href'])

In [4]:
# with open('image_pages.json', 'w') as f:
#     f.write(json.dumps(pages))

In [5]:
with open('image_pages.json', 'r') as f:
    image_pages = list(json.loads(f.read()).values())

In [6]:
api_image_urls = [img['image_href'] for img_page in image_pages for img in img_page]
print(len(api_image_urls), len(set(api_image_urls)))

361676 161223


Okay, got about .3 million images and about half of those are unique. Looks good, let's get 'em!

### 3. update API to save images to DO

### 4. setup for bulk image upload to DO

In [7]:
def hash_image(img):
    return imagehash.phash(img)
img = Image.open("temp.jpg")
f'{hash_image(img)}.jpg'

'9780f5773298c566.jpg'

In [8]:
def _save_image(url):
    # save image in temp folder
    r = requests.get(url, timeout=15)
    in_memory_file = io.BytesIO(r.content)
    im = Image.open(in_memory_file)
    hash_array = hash_image(im)
    ext = im.format.lower()
    fname = f'{folder}/{hash_array}.{ext}'
    im.save(fname)
    if VERBOSE_MODE:
        print('saved', fname)
    return f'{hash_array}.{ext}'
# _save_image('https://i.kym-cdn.com/photos/images/newsfeed/000/008/330/funny_cat_pictures_144.jpg')

### 5. run for all images

Set up Digital Ocean

In [9]:
count = 0
failures = {}

This is slow code which gets the images from their original URLs and puts them on DO, but only needs to be run once.

In [10]:
class Storage:
    def __init__(self):
        self.start = time.time()
        self._lock = threading.Lock()
        self.failures = {}
        self.errors = {}
        self.count = 0
        self.url_to_hash = {}
        
    def add_failure(self, image_id, url, e):
        with self._lock:
            self.failures[image_id] = url
            self.errors[image_id] = e
            if VERBOSE_MODE:
                print("error", e)

    def increment_count(self):
        with self._lock:
            self.count += 1
            
    def save_image(self, url):
        fname = _save_image(url)
        with self._lock:
            self.url_to_hash[url] = fname
            
    def save_mapping(self):
        fname = f'url_to_hash_{round(time.time())}.json'
        with self._lock:
            with open(fname, 'w') as f:
                f.write(json.dumps(self.url_to_hash))
        return fname
    
    def print_status(self):
        with self._lock:
            print(f"processing {self.count} took {round((time.time()-self.start)/60, 1)} minutes. \
{round(100*(len(self.failures)/self.count), 1)}% failure rate & {len(self.failures)} failures, {round((time.time()-self.start)/self.count, 2)} s per, ts {time.time()}")

storage = Storage()
def worker():
    while True:
        image_id, url = q.get()
        if url is None:
            print('x', end=' ')
            q.task_done()
            break
        
        storage.increment_count()
        
        try:
            storage.save_image(url)
        except Exception as e:
            storage.add_failure(image_id, url, e)
        print_freq = 100
        if VERBOSE_MODE:
            print_freq = 10
        if not SHORT:
            print_freq = 500
        if random.randint(0,print_freq) == 0:
            storage.print_status()
        q.task_done()

Use the cached responses from formerly requesting all the images from the API.

In [11]:
# set up threading and queueing infra
q = queue.Queue()
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

# iterating over all images in DB
start_idx = 0
if SHORT:
    start_idx = random.randint(5,50)
if RANGE[0] != None:
    start_idx = RANGE[0]
if RANGE[1] != None:
    end_idx = RANGE[1]
else:
    end_idx = len(image_pages)
for j in image_pages[start_idx:end_idx]:
    for img in j:
        count += 1
        # make filename from url
        if img['image_href'] is not None:
            url = img['image_href']
        elif img['wordpress_attachment_file_path'] is not None:
            url = 'http://firewallcafe.com' + img['wordpress_attachment_file_path']
        else:
            continue
        # put the url into the queue for the workers
        q.put((img['image_id'], url))
    if SHORT:
        print("skipping the rest of the pages because this is a test")
        break
print("\ndone with /image queries\ntelling threads to stop")
for i in range(num_worker_threads):
    q.put((None,None))
print('waiting')
for t in threads:
    t.join(timeout=3*60)
mapping_fname = storage.save_mapping()
print("saved to", mapping_fname)


done with /image queries
telling threads to stop
waiting
processing 719 took 0.1 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122497.980612
processing 1022 took 0.2 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122501.0673604
processing 2082 took 0.4 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122511.5273967
processing 2489 took 0.4 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122515.3634527
processing 2599 took 0.4 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122516.4390888
processing 2636 took 0.4 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122516.7991254
processing 2686 took 0.5 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122517.284828
processing 2772 took 0.5 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122518.1385815
processing 3578 took 0.6 minutes. 0.0% failure rate & 0 failures, 0.01 s per, ts 1618122526.2459843
processing 3684 took 0.6 minutes. 0.0% failur

C:\Users\rowan\anaconda3\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


processing 164499 took 30.2 minutes. 0.0% failure rate & 33 failures, 0.01 s per, ts 1618124301.4435785
processing 165283 took 32.4 minutes. 0.1% failure rate & 145 failures, 0.01 s per, ts 1618124435.5448728


C:\Users\rowan\anaconda3\lib\site-packages\PIL\Image.py:2929: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


processing 166465 took 36.2 minutes. 0.2% failure rate & 330 failures, 0.01 s per, ts 1618124663.1704326
processing 166600 took 36.5 minutes. 0.2% failure rate & 350 failures, 0.01 s per, ts 1618124682.841308
processing 167009 took 37.8 minutes. 0.2% failure rate & 413 failures, 0.01 s per, ts 1618124758.7031612
processing 168084 took 40.8 minutes. 0.3% failure rate & 579 failures, 0.01 s per, ts 1618124935.4604037
processing 168223 took 41.3 minutes. 0.4% failure rate & 598 failures, 0.01 s per, ts 1618124966.3299487
processing 169561 took 46.8 minutes. 0.5% failure rate & 840 failures, 0.02 s per, ts 1618125296.775748
processing 169764 took 47.6 minutes. 0.5% failure rate & 870 failures, 0.02 s per, ts 1618125345.5458918
processing 169922 took 48.1 minutes. 0.5% failure rate & 899 failures, 0.02 s per, ts 1618125375.2273517
processing 169952 took 48.2 minutes. 0.5% failure rate & 903 failures, 0.02 s per, ts 1618125380.6328995
processing 171092 took 51.0 minutes. 0.7% failure rate & 

C:\Users\rowan\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 34 bytes but only got 20. Skipping tag 42036
  warnings.warn(
C:\Users\rowan\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


processing 180692 took 78.0 minutes. 1.4% failure rate & 2509 failures, 0.03 s per, ts 1618127167.2367122
processing 180993 took 78.8 minutes. 1.4% failure rate & 2562 failures, 0.03 s per, ts 1618127216.0320628
processing 181151 took 79.3 minutes. 1.4% failure rate & 2595 failures, 0.03 s per, ts 1618127247.3494384
processing 181470 took 80.6 minutes. 1.5% failure rate & 2638 failures, 0.03 s per, ts 1618127324.3446307
processing 182029 took 82.1 minutes. 1.5% failure rate & 2717 failures, 0.03 s per, ts 1618127415.869392
processing 182500 took 83.3 minutes. 1.5% failure rate & 2786 failures, 0.03 s per, ts 1618127487.7040124
processing 183656 took 87.0 minutes. 1.6% failure rate & 2958 failures, 0.03 s per, ts 1618127711.8735435
processing 183807 took 87.6 minutes. 1.6% failure rate & 2991 failures, 0.03 s per, ts 1618127748.9652636
processing 184859 took 91.7 minutes. 1.7% failure rate & 3188 failures, 0.03 s per, ts 1618127989.7493973
processing 185423 took 93.7 minutes. 1.8% failu

C:\Users\rowan\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 4. 
  warnings.warn(str(msg))


processing 202515 took 140.5 minutes. 3.0% failure rate & 6075 failures, 0.04 s per, ts 1618130920.445903
processing 202923 took 141.6 minutes. 3.0% failure rate & 6176 failures, 0.04 s per, ts 1618130986.7774377
processing 203706 took 144.4 minutes. 3.1% failure rate & 6346 failures, 0.04 s per, ts 1618131151.3742428
processing 203804 took 144.7 minutes. 3.1% failure rate & 6365 failures, 0.04 s per, ts 1618131173.3877654
processing 204585 took 147.6 minutes. 3.2% failure rate & 6502 failures, 0.04 s per, ts 1618131344.7359045
processing 205052 took 149.2 minutes. 3.2% failure rate & 6595 failures, 0.04 s per, ts 1618131439.5949085
saved to url_to_hash_1618131496.json


In [33]:
mapping_fname = storage.save_mapping()
print("saved to", mapping_fname)
storage.print_status()

saved to url_to_hash_1618157001.json
processing 361676 took 575.3 minutes. 22.9% failure rate & 82903 failures, 0.1 s per, ts 1618157007.5260353


Check all image URLs in the list against their corresponding filenames in the folder.

In [34]:
len(storage.url_to_hash)

101693

In [35]:
saved_images = set(os.listdir(folder))

In [36]:
len(saved_images)

94884

Check if we were able to get all the images we need by comparing the files we have saved to the original URL, and making sure there is coverage there.

In [37]:
found_count = 0
missing = []
saved_urls = set(storage.url_to_hash.keys())
for image_page in image_pages:
    for image in image_page:
        if image['image_href'] in saved_urls:
            found_count += 1
        else:
            missing.append(image)
print(f"found {found_count} ({round(100*found_count/(found_count+len(missing)), 2)}%), missing {len(missing)}")

found 267437 (73.94%), missing 94239


In [16]:
non_jpgs = [fname for fname in saved_images if fname[-4:] != '.jpg' and fname[-5:] != '.jpeg']

In [38]:
len(non_jpgs)/len(saved_images)

0.026021246996332365

Sanity-check a random selection of images to make sure they correspond to files we actually have.

In [39]:
sample = random.sample(storage.url_to_hash.items(), 5)
originals = [el[0] for el in sample]
copies = [el[1] for el in sample]

In [40]:
ipyplot.plot_images(originals)

In [41]:
ipyplot.plot_images([f'{folder}/{fname}' for fname in copies])

That checks out. Let's look at the images which are most referred to in the database.

In [42]:
api_image_urls = [img['image_href'] for img_page in image_pages for img in img_page]

In [43]:
len(api_image_urls)

361676

In [44]:
random.sample(api_image_urls, 10)

['http://img2.imgtn.bdimg.com/it/u=392972713,3849171745&fm=21&gp=0.jpg',
 'https://firewall-cafe-space.nyc3.digitaloceanspaces.com/images/hashed/c5d5da98476585c3.jpg',
 'http://www.bohemia-apartments.com/wp-content/uploads/2014/09/warhol4.jpg',
 'https://firewall-cafe-space.nyc3.digitaloceanspaces.com/images/hashed/b830876d838f8fc6.jpg',
 'https://firewall-cafe-space.nyc3.digitaloceanspaces.com/images/hashed/8000000000000000.gif',
 'https://firewall-cafe-space.nyc3.digitaloceanspaces.com/images/hashed/fbecd2c88e161992.jpg',
 None,
 'https://firewall-cafe-space.nyc3.digitaloceanspaces.com/images/hashed/b503786f05f31e70.jpg',
 'http://cdn.wegotthiscovered.com/wp-content/uploads/Silent-Hills-670x335.jpg',
 'http://img4.imgtn.bdimg.com/it/u=3630604270,565219518&fm=21&gp=0.jpg']

In [45]:
len(set(api_image_urls))

161223

Which images are duplicated the most in the API's database? 

In [46]:
url_counts = defaultdict(int)
for url in api_image_urls:
    url_counts[url] += 1
most_redundant_images = list(sorted(url_counts.items(), key=lambda el: el[1], reverse=True))

In [47]:
ipyplot.plot_images([f'{el[0]}' for el in most_redundant_images], [el[1] for el in most_redundant_images[:10]])